# **Excavating New Datasets via GeoSpatial Insights from Datalakes**


Eric Martin <br>
Federico Larrieu <br>
CS 555 Distributed Systems <br>
Colorado State University <br>
Spring 2024 <br>

### Objective

    • Perform analytics over a large-scale temporal network

### Overview
In this assignment, I will perform an analysis of a continuously evolving temporal network. Large-scale networks are observed in many different sociological and scientific settings such as computer networks, networks of social media, academic/technical citation networks and hyperlink networks. To understand such networks, there have been several properties of interest based primarily on two key measurements: the degrees of nodes and the shortest distances between pairs of nodes. The node-to-node distances often infer the graph’s diameter, which is the maximum shortest distance among all the connected pairs of nodes. 

Most of the large networks evolve over time by adding new members/items and relationships between them or removing some of them. In the traditional temporal network analysis, there are two major hypotheses. 

    (a) the average node degree in the network remains constant over time.
    (Or the number of edges grows linearly in the number of nodes.). 

    (b) the diameter is a slowly growing function of the network size. 

    How are these hypothesis (a) and (b) reflected in real-world data?

In this assignment, I measure fundamental network properties with a 
citation network and investigate how they evolve. I will perform the following computations using Apache Spark.

### Dataset

The dataset for this assignment is the arXiv citation graph ( J. Gehrke, P. Ginsparg, and J. M. Kleinberg. Overview of the 2003 kdd cup. SIGKDD Explorations, 5(2):149–151, 2003) that covers papers published in the period from January 1993 to April 2003 (11 years). 

Please note that the dataset for the last year (2003) is incomplete and does not represent the entire year.

## Task 1: Exercises on the basic Spark features

In this task, I will practice with the key features of the Apache Spark.

    (1) Count the number of unique published papers per year - create an output file with the number of papers published each year.

    (2) Count the number of edges (citations) generated per year - create an output file with the number of citations added each year.


First I am going to transfer my two datasets to the HDFS file system. I will use the following commands to transfer the files to the HDFS file system.

1) Start HDFS on NameNode
    ```bash
    start-dfs.sh
    ```
2) Start YARN on ResourceManager
    ```bash
    start-yarn.sh
    ```
3) Start Spark on the NameNode 
    start-master.sh
    start-workers.sh
    ```
4) Transfer the files to HDFS
    ```bash
        hadoop fs -mkdir /pa1
        hadoop fs -mkdir /pa1/input
        hadoop fs -put cs535/PA1/citations-redo.txt /pa1/input
        hadoop fs -put cs535/PA1/published-dates-redo.txt /pa1/input
    ```
5) Verify the files are in HDFS
    5.1) Set up SSH with tunneling
    ```bash
        ssh -L 8080:localhost:8080 ebmartin@hartford.cs.colostate.edu
    ```
    5.2) Find HDFS web address 
    ```bash
        cd hadoopConf
        vim hdfs-site.xml
    ```
    5.3) Find the following:
    ```xml
        <property>
            <name>dfs.namenode.http-address</name>
            <value>hartford.cs.colostate.edu:30182</value>
            <description>Location of the DFS web UI</description>
        </property>
    ``````
    5.4) Open local web browser and go to HDFS web address (http://<namenode>:<port>)
    ```bash
        http://hartford.cs.colostate.edu:30182/
    ```
    5.5) Verify files are in HDFS
    ```bash
        http://hartford.cs.colostate.edu:30182/explorer.html#/pa1/input
    ```
6) Check the YARN web portal to see if the Spark application is running (http://<resource_manager_host>:<port>)
    ```bash
        http://honolulu.cs.colostate.edu:30194/
    ```
7) Check the Spark web portal to see if the Spark application is running (http://<SPARK_MASTER_IP>:<SPARK_MASTER_WEBUI_PORT>)
    ```bash
        http://hartford.cs.colostate.edu:30197/
    ```
    *Note:*
    *I wrote all this so I can reference how I did it in future assignments.*

### Load data into Spark

In [1]:
import sys
sys.path.append("/usr/local/python-env/py39/lib/python3.9/site-packages")

import pyspark
print(pyspark.__version__)


print(sys.executable)

3.5.0
/usr/bin/python3.9


### Initialze a SparkSession

Initialize a test session to ensure the SparkSession is working properly. This will connect to the resource manager node that is running the YARN cluster. If we visit the YARN web portal, we can see that the Spark application is running.

Ensuring the pyspark library is being accessed from my local usr directory.


In [2]:
import os
os.environ['PYSPARK_PYTHON'] = '/usr/bin/python3.9'

In [3]:
import pkg_resources

sedona_version = pkg_resources.get_distribution("apache-sedona").version
print(f"Apache Sedona version: {sedona_version}")

Apache Sedona version: 1.5.1


In [4]:
print(os.environ['SPARK_HOME'])
print(os.environ['PYSPARK_PYTHON'])

/usr/local/spark/latest
/usr/bin/python3.9


## Now to make the app

In [5]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import split
from pyspark.sql.functions import col
from pyspark.sql import Row
from pyspark.sql.types import IntegerType, DateType
from pyspark.sql.functions import year  # used to extract year from date, could do this manually as well
from pyspark.sql import Window
from pyspark.sql.functions import sum as pyspark_sum


from pyspark.sql import SparkSession
from sedona.register import SedonaRegistrator
from sedona.utils import SedonaKryoRegistrator, KryoSerializer
from sedona.spark import *
import geopandas as gpd

spark = SparkSession \
    .builder \
    .appName('GeoSpatialQueries_eric') \
    .master('spark://hartford:30196') \
    .config("spark.yarn.resourcemanager.address", "honolulu.cs.colostate.edu:30190") \
    .config("spark.serializer", KryoSerializer.getName) \
    .config("spark.kryo.registrator", SedonaKryoRegistrator.getName) \
    .config('spark.jars.packages',
            'org.apache.sedona:sedona-spark-3.5_2.12:1.5.1,'
            'org.datasyslab:geotools-wrapper:1.5.1-28.2') \
    .config('spark.jars.repositories', 'https://artifacts.unidata.ucar.edu/repository/unidata-all') \
    .getOrCreate()

# Set log level to DEBUG
spark.sparkContext.setLogLevel("ERROR")

sedona = SedonaContext.create(spark)
SedonaRegistrator.registerAll(spark)

# create a logger
logger = spark._jvm.org.apache.log4j.LogManager.getLogger(__name__)
logger.info("Pyspark initialized...")

# IF YOU WANT TO RUN THE TEST, SET isTest = True
isTest = False

Skipping SedonaKepler import, verify if keplergl is installed


https://artifacts.unidata.ucar.edu/repository/unidata-all added as a remote repository with the name: repo-1
Ivy Default Cache set to: /s/chopin/l/grad/ebmartin/.ivy2/cache
The jars for the packages stored in: /s/chopin/l/grad/ebmartin/.ivy2/jars
org.apache.sedona#sedona-spark-3.5_2.12 added as a dependency
org.datasyslab#geotools-wrapper added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-0580c216-7ddb-43cc-b8ad-89bad055763f;1.0
	confs: [default]
	found org.apache.sedona#sedona-spark-3.5_2.12;1.5.1 in central
	found org.apache.sedona#sedona-common;1.5.1 in central
	found org.apache.commons#commons-math3;3.6.1 in local-m2-cache
	found org.locationtech.jts#jts-core;1.19.0 in central
	found org.wololo#jts2geojson;0.16.1 in central
	found org.locationtech.spatial4j#spatial4j;0.8 in central
	found com.google.geometry#s2-geometry;2.0.0 in central
	found com.google.guava#guava;25.1-jre in central
	found com.google.code.findbugs#jsr305;3.0.2 in local-m2-cac

:: loading settings :: url = jar:file:/usr/local/spark/3.5.0-with-hadoop3.3/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found org.checkerframework#checker-qual;2.0.0 in central
	found com.google.errorprone#error_prone_annotations;2.1.3 in central
	found com.google.j2objc#j2objc-annotations;1.1 in local-m2-cache
	found org.codehaus.mojo#animal-sniffer-annotations;1.14 in central
	found com.uber#h3;4.1.1 in central
	found net.sf.geographiclib#GeographicLib-Java;1.52 in central
	found com.github.ben-manes.caffeine#caffeine;2.9.2 in central
	found org.checkerframework#checker-qual;3.10.0 in central
	found com.google.errorprone#error_prone_annotations;2.5.1 in central
	found org.apache.sedona#sedona-spark-common-3.5_2.12;1.5.1 in central
	found commons-lang#commons-lang;2.6 in local-m2-cache
	found org.scala-lang.modules#scala-collection-compat_2.12;2.5.0 in central
	found org.beryx#awt-color-factory;1.0.0 in central
	found org.datasyslab#geotools-wrapper;1.5.1-28.2 in central
:: resolution report :: resolve 300ms :: artifacts dl 14ms
	:: modules in use:
	com.github.ben-manes.caffeine#caffeine;2.9.2 from ce

## Load the datasets

Got the code for this from https://sedona.apache.org/1.5.1/tutorial/sql/

Load GeoJSON using Spark JSON Data Source:

Spark SQL's built-in JSON data source supports reading GeoJSON data. To ensure proper parsing of the geometry property, we can define a schema with the geometry property set to type 'string'. This prevents Spark from interpreting the property and allows us to use the ST_GeomFromGeoJSON function for accurate geometry parsing.

```python
schema = "type string, crs string, totalFeatures long, features array<struct<type string, geometry string, properties map<string, string>>>";
(sedona.read.json(geojson_path, schema=schema)
    .selectExpr("explode(features) as features") # Explode the envelope to get one feature per row.
    .select("features.*") # Unpack the features struct.
    .withColumn("geometry", f.expr("ST_GeomFromGeoJSON(geometry)")) # Convert the geometry string.
    .printSchema())
```

In [6]:
# Import the necessary module from py4j to interact with JVM
from py4j.java_gateway import java_import

# Import the Path class from Hadoop. This class is used to handle file paths in Hadoop.
java_import(spark._jvm, 'org.apache.hadoop.fs.Path')

# Define a function to recursively get all .json and .geojson files in a directory and its subdirectories
def get_files_recursive(path):
    # Use the listStatus method of the FileSystem class to get an array of FileStatus objects
    # Each FileStatus object represents a file or directory in the given path
    file_status_arr = fs.listStatus(spark._jvm.Path(path))
    
    # Initialize an empty list to hold the file paths
    file_paths = []
    
    # Loop through each FileStatus object in the array
    for file_status in file_status_arr:
        # If the FileStatus object represents a directory
        if file_status.isDirectory():
            # Call the get_files_recursive function with the directory path
            # This is a recursive call, which means the function calls itself
            # Add the returned file paths to the file_paths list
            file_paths += get_files_recursive(file_status.getPath().toString())
        # If the FileStatus object represents a file that ends with .json or .geojson
        elif file_status.getPath().getName().endswith(('.json', '.geojson')):
            # Add the file path to the file_paths list
            file_paths.append(file_status.getPath().toString())
    
    # Return the list of file paths
    return file_paths

In [7]:

# Initialize a Hadoop file system 
fs = spark._jvm.org.apache.hadoop.fs.FileSystem.get(spark._jsc.hadoopConfiguration())

# Directory containing the files
json_directory = "hdfs://hartford.cs.colostate.edu:30181/geospatial/input/"

# Define the schema for the GeoJSON data
geojsonSchema = "type string, crs string, totalFeatures long, features array<struct<type string, geometry string, properties map<string, string>>>"


if isTest:

    # Path to the GeoJSON file
    geojson_path = "hdfs://hartford.cs.colostate.edu:30181/geospatial/input/cb_2018_us_state_20m.json"

    # Read the GeoJSON file using the defined schema using sedona into a spark dataframe
    state_boundaries_sedona = spark.read.schema(geojsonSchema).json(geojson_path, multiLine=True)
    
    # Explode the features array to create a row for each feature and select the columns
    state_boundaries_sedona = (state_boundaries_sedona
                               .select(F.explode("features").alias("features"))
                               .select("features.*")
                               # Use Sedona's ST_GeomFromGeoJSON function to convert the geometry string to a geometry object
                               .withColumn("geometry", F.expr("ST_GeomFromGeoJSON(geometry)"))
                              )

else:
    # Get a list of the JSON and GeoJSON files in the directory and its subdirectories
    json_files = get_files_recursive(json_directory)

    # Create a dictionary to hold the DataFrames
    json_dataset_dataframes = {}

    # Define the current and desired EPSG codes
    current_epsg = "EPSG:3857"  # Web Mercator
    desired_epsg = "EPSG:4326"  # WGS84

    # Load each JSON file into a DataFrame and store it in the dictionary
    for file_path in json_files:
        file_name = file_path.split('/')[-1]
        
        # Print the file path
        print(f"Processing file: {file_path}")
        
        # Read the GeoJSON file using the defined schema using sedona into a spark dataframe
        df = spark.read.schema(geojsonSchema).json(file_path, multiLine=True)
        
        # Explode the features array to create a row for each feature and select the columns
        df = (df
            .select(F.explode("features").alias("features"))
            .select("features.*")
            # Use Sedona's ST_GeomFromGeoJSON function to convert the geometry string to a geometry object
            .withColumn("geometry", F.expr("ST_GeomFromGeoJSON(geometry)"))
            # Transform the 'geometry' column from the current EPSG code to the desired EPSG code
            .withColumn("geometry", F.expr(f"ST_Transform(geometry, '{current_epsg}', '{desired_epsg}')"))
            )
        
        json_dataset_dataframes[file_name] = df

Processing file: hdfs://hartford.cs.colostate.edu:30181/geospatial/input/AgriculturalArea/AgriculturalArea.geojson
Processing file: hdfs://hartford.cs.colostate.edu:30181/geospatial/input/CityBoundaries/CityBoundaries.geojson
Processing file: hdfs://hartford.cs.colostate.edu:30181/geospatial/input/CountryTerritories/CountryTerritories.geojson
Processing file: hdfs://hartford.cs.colostate.edu:30181/geospatial/input/EcoRegions/EcoRegions.geojson
Processing file: hdfs://hartford.cs.colostate.edu:30181/geospatial/input/GeographicRegions/GeographicRegions.geojson
Processing file: hdfs://hartford.cs.colostate.edu:30181/geospatial/input/GeologicalFeatures/Areas_diapiric_structures.geojson
Processing file: hdfs://hartford.cs.colostate.edu:30181/geospatial/input/GeologicalFeatures/Bathymetry.geojson
Processing file: hdfs://hartford.cs.colostate.edu:30181/geospatial/input/GeologicalFeatures/Calderas.geojson
Processing file: hdfs://hartford.cs.colostate.edu:30181/geospatial/input/GeologicalFeatur

## We can see that the datasets are not in a workable format

In [8]:
# print the schemas

if (isTest):
    print("State Boundaries Sedona Schema:")
    state_boundaries_sedona.printSchema()
else:
    for key, value in json_dataset_dataframes.items():
        print(f"{key} Schema:")
        value.printSchema()

AgriculturalArea.geojson Schema:
root
 |-- type: string (nullable = true)
 |-- geometry: geometry (nullable = true)
 |-- properties: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)

CityBoundaries.geojson Schema:
root
 |-- type: string (nullable = true)
 |-- geometry: geometry (nullable = true)
 |-- properties: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)

CountryTerritories.geojson Schema:
root
 |-- type: string (nullable = true)
 |-- geometry: geometry (nullable = true)
 |-- properties: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)

EcoRegions.geojson Schema:
root
 |-- type: string (nullable = true)
 |-- geometry: geometry (nullable = true)
 |-- properties: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)

GeographicRegions.geojson Schema:
root
 |-- type: string (nullable = true)
 |-- geometry: 

In [9]:
# View the first 5 rows of the state_boundaries_sedona DataFrame

if (isTest):
    state_boundaries_sedona.show(5, truncate=False)

## Running Spatial Queries

https://sedona.apache.org/1.5.1/api/sql/Function/

### Range Query

This example demonstrates how to perform a range query using ST_Contains to find geometries within a specified polygon:

In [10]:
# Define a polygon using ST_PolygonFromEnvelope and perform a range query

bbox_polygon = "ST_PolygonFromEnvelope(-79.5, 37.9, -75.6, 39.8)"

# Perform the range query to find features within the bounding box
contained_features = state_boundaries_sedona.filter(
    F.expr(f"ST_Contains({bbox_polygon}, geometry)")
)

# Show results
contained_features.show()

NameError: name 'state_boundaries_sedona' is not defined

## KNN Query

This example demonstrates how to perform a k-nearest neighbors (KNN) query using ST_Distance to find the k nearest geometries to a specified point:

In [ ]:
from pyspark.sql import functions as F

# Calculate the center of the bounding box and create a WKT representation of the point
center_longitude = (-79.5 + -75.6) / 2
center_latitude = (37.9 + 39.8) / 2
center_point_wkt = f"POINT({center_longitude} {center_latitude})"

# Perform the KNN query using ST_Distance to calculate the distance to the center point
knnQueryResult = state_boundaries_sedona.select(
    # Access the 'NAME' from the 'properties' map
    F.col("properties").getItem("NAME").alias("NAME"),
    F.expr(f"ST_Distance(ST_GeomFromWKT('{center_point_wkt}'), geometry)").alias("distance")
).orderBy("distance").limit(5)

knnQueryResult.show()



+--------------------+-------------------+
|                NAME|           distance|
+--------------------+-------------------+
|            Virginia|                0.0|
|            Maryland| 0.2425364842513449|
|       West Virginia|0.39633443061384227|
|District of Columbia|0.43843022219048333|
|        Pennsylvania| 0.8705736535332227|
+--------------------+-------------------+



In [ ]:
state_boundaries_sedona.show()

+-------+--------------------+--------------------+
|   type|            geometry|          properties|
+-------+--------------------+--------------------+
|Feature|MULTIPOLYGON (((-...|{STATEFP -> 24, S...|
|Feature|POLYGON ((-96.621...|{STATEFP -> 19, S...|
|Feature|POLYGON ((-75.773...|{STATEFP -> 10, S...|
|Feature|MULTIPOLYGON (((-...|{STATEFP -> 39, S...|
|Feature|POLYGON ((-80.519...|{STATEFP -> 42, S...|
|Feature|POLYGON ((-104.05...|{STATEFP -> 31, S...|
|Feature|MULTIPOLYGON (((-...|{STATEFP -> 53, S...|
|Feature|MULTIPOLYGON (((-...|{STATEFP -> 72, S...|
|Feature|POLYGON ((-88.468...|{STATEFP -> 01, S...|
|Feature|POLYGON ((-94.617...|{STATEFP -> 05, S...|
|Feature|POLYGON ((-109.04...|{STATEFP -> 35, S...|
|Feature|POLYGON ((-106.62...|{STATEFP -> 48, S...|
|Feature|MULTIPOLYGON (((-...|{STATEFP -> 06, S...|
|Feature|POLYGON ((-89.544...|{STATEFP -> 21, S...|
|Feature|POLYGON ((-85.605...|{STATEFP -> 13, S...|
|Feature|MULTIPOLYGON (((-...|{STATEFP -> 55, S...|
|Feature|POL

In [11]:
# Define the precision for the GeoHash
precision = 17

# Apply the ST_GeoHash function to each DataFrame in the dictionary
for file_name, df in json_dataset_dataframes.items():
    # Add a new column 'geohash' to the DataFrame
    # The new column is the GeoHash of the 'geometry' column with the given precision
    df = df.withColumn('geohash', F.expr(f"ST_GeoHash(geometry, {precision})"))
    
    # Update the DataFrame in the dictionary
    json_dataset_dataframes[file_name] = df

In [12]:
# Loop through each DataFrame in the dictionary
for file_name, df in json_dataset_dataframes.items():
    # Print the file name
    print(f"File: {file_name}")
    
    # Show the first few rows of the DataFrame
    df.show()

File: AgriculturalArea.geojson


+-------+--------------------+--------------------+-----------------+
|   type|            geometry|          properties|          geohash|
+-------+--------------------+--------------------+-----------------+
|Feature|POLYGON ((-86.411...|{atlas_stco -> 01...|djf2gyhbv2zcvfuq5|
|Feature|POLYGON ((-87.765...|{atlas_stco -> 01...|dj3xh3s29uwyntq0h|
|Feature|POLYGON ((-85.051...|{atlas_stco -> 01...|djem2sjezsn8d3f48|
|Feature|POLYGON ((-86.881...|{atlas_stco -> 01...|djf5bys527zx3dc1w|
|Feature|POLYGON ((-86.303...|{atlas_stco -> 01...|dn43kxg2ne76f5uqj|
|Feature|POLYGON ((-85.433...|{atlas_stco -> 01...|djen9drdpb0td407b|
|Feature|POLYGON ((-86.499...|{atlas_stco -> 01...|djdkedfefunffsg28|
|Feature|POLYGON ((-85.530...|{atlas_stco -> 01...|dn4bndnzcsk6uqfwr|
|Feature|POLYGON ((-85.186...|{atlas_stco -> 01...|djg738ee4883qth40|
|Feature|POLYGON ((-85.463...|{atlas_stco -> 01...|dn54exu7y6bqterch|
|Feature|POLYGON ((-86.517...|{atlas_stco -> 01...|djf6gpjngyymyuzwy|
|Feature|POLYGON ((-

+-------+--------------------+--------------------+-----------------+
|   type|            geometry|          properties|          geohash|
+-------+--------------------+--------------------+-----------------+
|Feature|POLYGON ((-98.181...|{NAME -> Pharr, C...|9udhv9gc9gecvqr6b|
|Feature|POLYGON ((-98.226...|{NAME -> McAllen,...|9udjhn79s44yyf1ck|
|Feature|POLYGON ((-98.138...|{NAME -> Edinburg...|9udjyhewy9e3jpdst|
|Feature|POLYGON ((-99.627...|{NAME -> Laredo, ...|9uchybk9yh4ghr3qh|
|Feature|POLYGON ((-98.335...|{NAME -> Mission,...|9udj43639h5p1xxn3|
|Feature|MULTIPOLYGON (((-...|{NAME -> San Anto...|9v1zwnv74ny5kz9dp|
|Feature|POLYGON ((-97.615...|{NAME -> Round Ro...|9v6t9mjhcjk8p3pnp|
|Feature|MULTIPOLYGON (((-...|{NAME -> Austin, ...|9v6krtket1p5txews|
|Feature|MULTIPOLYGON (((-...|{NAME -> Killeen,...|9vd88s51j8bfr5cc5|
|Feature|POLYGON ((-97.437...|{NAME -> Brownsvi...|9udsn62mw79ru7q95|
|Feature|POLYGON ((-100.45...|{NAME -> San Ange...|9v8df609xcq644ts7|
|Feature|POLYGON ((-

+-------+--------------------+--------------------+-----------------+
|   type|            geometry|          properties|          geohash|
+-------+--------------------+--------------------+-----------------+
|Feature|POLYGON ((0.00029...|{geo_point_2d -> ...|s0000001ph9sp9kfk|
|Feature|POLYGON ((0.00008...|{geo_point_2d -> ...|s00000043wrhgd1ze|
|Feature|POLYGON ((-0.0000...|{geo_point_2d -> ...|ebpbpbpfxd3tppqpc|
|Feature|POLYGON ((0.00015...|{geo_point_2d -> ...|s00000046zckv90ds|
|Feature|POLYGON ((-0.0000...|{geo_point_2d -> ...|ebpbpbpfn672zmu2u|
|Feature|POLYGON ((0.00005...|{geo_point_2d -> ...|s0000004973j3v8cw|
|Feature|POLYGON ((0.00078...|{geo_point_2d -> ...|s0000006w02m0ug1k|
|Feature|POLYGON ((-0.0006...|{geo_point_2d -> ...|ebpbpbp89kgqfgbxm|
|Feature|POLYGON ((0.00011...|{geo_point_2d -> ...|s00000044w5mkyz6h|
|Feature|MULTIPOLYGON (((0...|{geo_point_2d -> ...|s000000045fjz8jsn|
|Feature|POLYGON ((-0.0006...|{geo_point_2d -> ...|ebpbpbp2zy64rw3bk|
|Feature|MULTIPOLYGO

+-------+--------------------+--------------------+-----------------+
|   type|            geometry|          properties|          geohash|
+-------+--------------------+--------------------+-----------------+
|Feature|POLYGON ((-20.518...|{US_L3CODE -> 1, ...|edcx39wzyh0rw2wv3|
|Feature|POLYGON ((-20.902...|{US_L3CODE -> 1, ...|ee14sq49zdwvdswf5|
|Feature|POLYGON ((-20.901...|{US_L3CODE -> 1, ...|ee14sr41kemvhhyee|
|Feature|POLYGON ((-20.915...|{US_L3CODE -> 1, ...|ee14uhcuq9q0t79me|
|Feature|POLYGON ((-20.954...|{US_L3CODE -> 1, ...|ee155jt16k77cycup|
|Feature|POLYGON ((-20.955...|{US_L3CODE -> 1, ...|ee155nhdytpfp852m|
|Feature|POLYGON ((-20.954...|{US_L3CODE -> 1, ...|ee155nj7xwwknfgyw|
|Feature|POLYGON ((-20.959...|{US_L3CODE -> 1, ...|ee155p1em6ys2gsvk|
|Feature|POLYGON ((-20.741...|{US_L3CODE -> 1, ...|ee1k3sn8m8ejswy4p|
|Feature|POLYGON ((-20.759...|{US_L3CODE -> 1, ...|ee1jzx3bej7k47d2n|
|Feature|POLYGON ((-20.758...|{US_L3CODE -> 1, ...|ee1npe5gm5er8qt2p|
|Feature|POLYGON ((-

+-------+--------------------+--------------------+-----------------+
|   type|            geometry|          properties|          geohash|
+-------+--------------------+--------------------+-----------------+
|Feature|LINESTRING (-21.6...|{Shape_Leng -> 2....|eqvmurp1s9wk843kk|
|Feature|LINESTRING (12.94...|{Shape_Leng -> 2....|ewr2mz5h211j65se7|
|Feature|LINESTRING (13.91...|{Shape_Leng -> 1....|ewr8kx0hr35vk5e7q|
|Feature|LINESTRING (16.96...|{Shape_Leng -> 2....|ey22rx4s7c5mr5e7k|
|Feature|LINESTRING (39.94...|{Shape_Leng -> 3....|sqqyp8hk3sgcw9dg3|
|Feature|LINESTRING (39.94...|{Shape_Leng -> 3....|sqrn2g7854y781wyw|
|Feature|LINESTRING (39.94...|{Shape_Leng -> 2....|sxh5kqjxrfn1qc23v|
|Feature|LINESTRING (39.94...|{Shape_Leng -> 2....|sxh8renumeznh52fn|
|Feature|LINESTRING (39.94...|{Shape_Leng -> 1....|swkzyyu9dcdb8upd4|
|Feature|LINESTRING (-17.2...|{Shape_Leng -> 30...|exkrqpwcxy7xpmk84|
|Feature|LINESTRING (-38.6...|{Shape_Leng -> 1....|ercgtxukcrck7e31k|
|Feature|LINESTRING 

+-------+--------------------+--------------------+-----------------+
|   type|            geometry|          properties|          geohash|
+-------+--------------------+--------------------+-----------------+
|Feature|LINESTRING (-1.98...|{Shape_Leng -> 33...|gfq9t4e745358cx17|
|Feature|LINESTRING (5.664...|{Shape_Leng -> 78...|sh7vxdkx5n77knhpc|
|Feature|LINESTRING (5.807...|{Shape_Leng -> 14...|shknkrm318gxpk9uq|
|Feature|LINESTRING (5.916...|{Shape_Leng -> 30...|shknrcn8f538fq3ed|
|Feature|LINESTRING (6.181...|{Shape_Leng -> 78...|shkmvry8hd3y5zzfp|
|Feature|LINESTRING (6.155...|{Shape_Leng -> 57...|shkqkdjrh28pmj556|
|Feature|LINESTRING (6.070...|{Shape_Leng -> 11...|shkqdupq089hhh22u|
|Feature|LINESTRING (6.070...|{Shape_Leng -> 17...|shkr7f9prfj20qm4b|
|Feature|LINESTRING (9.316...|{Shape_Leng -> 27...|shwdkznww32qqszkx|
|Feature|LINESTRING (9.104...|{Shape_Leng -> 10...|shw8vtwynxxvuctzx|
|Feature|LINESTRING (9.372...|{Shape_Leng -> 66...|shwdpqrub8rmm27kk|
|Feature|LINESTRING 

+-------+--------------------+--------------------+-----------------+
|   type|            geometry|          properties|          geohash|
+-------+--------------------+--------------------+-----------------+
|Feature|POINT (30.4060414...|{SHAPE_TYPE -> Do...|u8m85cj5bjgc88ujs|
|Feature|POINT (30.9265697...|{SHAPE_TYPE -> Do...|u8mbzz0p1kcf3r3mk|
|Feature|POINT (30.6843429...|{SHAPE_TYPE -> Do...|u8mb65retn8kvd9b4|
|Feature|POINT (30.2791350...|{SHAPE_TYPE -> Do...|u8jxch2x6crt91nkz|
|Feature|POINT (30.5065310...|{SHAPE_TYPE -> Do...|u8jsy1y1tswmbxgt1|
|Feature|POINT (30.5065634...|{SHAPE_TYPE -> Do...|u8jewnnpq2wmsfemw|
|Feature|POINT (32.0420266...|{SHAPE_TYPE -> Do...|u8wz94sk96dwzextx|
|Feature|POINT (32.1134005...|{SHAPE_TYPE -> Do...|u8wydb0k2mz528xvg|
|Feature|POINT (32.4900208...|{SHAPE_TYPE -> Do...|u8xng24w03g7m7huu|
|Feature|POINT (31.5584119...|{SHAPE_TYPE -> Do...|u8w7w5k4h73sc4rsr|
|Feature|POINT (32.2512126...|{SHAPE_TYPE -> Do...|u8qgvgsx5srhs3yhz|
|Feature|POINT (31.8

+-------+--------------------+--------------------+-----------------+
|   type|            geometry|          properties|          geohash|
+-------+--------------------+--------------------+-----------------+
|Feature|LINESTRING (4.577...|{SLIP_DIR -> NULL...|shd9ex78fc8t5wtw9|
|Feature|LINESTRING (4.058...|{SLIP_DIR -> NULL...|shd393k5ppb03whbz|
|Feature|LINESTRING (-19.0...|{SLIP_DIR -> NULL...|ewf6jr182qmnjbnde|
|Feature|LINESTRING (-19.3...|{SLIP_DIR -> NULL...|ewf4qx8334qkc990r|
|Feature|LINESTRING (-19.6...|{SLIP_DIR -> NULL...|ewf4bsqkqbxjvfwnm|
|Feature|LINESTRING (-19.0...|{SLIP_DIR -> NULL...|ewf7meqcu1u674z6v|
|Feature|LINESTRING (-19.3...|{SLIP_DIR -> NULL...|ewdnyxnrcthgd7fd9|
|Feature|LINESTRING (-19.2...|{SLIP_DIR -> NULL...|ewdr67mkbv9s61nqm|
|Feature|LINESTRING (-19.3...|{SLIP_DIR -> NULL...|ewdpr7u8wjjfv3eg7|
|Feature|LINESTRING (-19.2...|{SLIP_DIR -> NULL...|ewdqgkzr5zupje05g|
|Feature|LINESTRING (-19.2...|{SLIP_DIR -> NULL...|ewdrg11248g89jbfn|
|Feature|LINESTRING 

+-------+--------------------+----------------+-----------------+
|   type|            geometry|      properties|          geohash|
+-------+--------------------+----------------+-----------------+
|Feature|POINT (-22.241921...| {OBJECTID -> 0}|gd2jmf5fjsv6ecn7g|
|Feature|POINT (-16.611828...| {OBJECTID -> 1}|exhhtyrtsqgh573gt|
|Feature|POINT (-16.493114...| {OBJECTID -> 2}|exhk8dq4d8dsgpyyg|
|Feature|POINT (18.8519986...| {OBJECTID -> 3}|smmqkzp853tpbrzc9|
|Feature|POINT (7.76133436...| {OBJECTID -> 4}|shmxbsedu4nqge4rd|
|Feature|POINT (4.52164832...| {OBJECTID -> 5}|sh5pqbuxdn3r5mq6t|
|Feature|POINT (4.47757233...| {OBJECTID -> 6}|sh7hjvhqwyrfvmmkq|
|Feature|POINT (4.28080609...| {OBJECTID -> 7}|sh7j17vnwhb2u5y0v|
|Feature|POINT (4.41620004...| {OBJECTID -> 8}|sh7jhqzf2ymmt9vyk|
|Feature|POINT (4.55000298...| {OBJECTID -> 9}|sh7jrd3pncb4uj7pf|
|Feature|POINT (5.65508131...|{OBJECTID -> 10}|shkhbttysvqr0nq7p|
|Feature|POINT (5.74795570...|{OBJECTID -> 11}|shkj4gcw9hf2veb2f|
|Feature|P